In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dataset= pd.read_csv("apple_dataset.csv")
dataset.tail(10)

In [ ]:
#checking any null values is present or not
x = dataset.isnull().sum()
x

In [ ]:
loss= dataset[['close']]
list_loss = np.array(loss)
list_loss.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
list_loss = scaler.fit_transform(list_loss)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(list_loss)

In [ ]:
n_train_data = int(len(list_loss)/1.6)
train_data = list_loss[:n_train_data]
n_test_data = (len(list_loss) - n_train_data)
test_data =  list_loss[n_train_data:]


In [ ]:
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 100
X_train,Y_train = create_dataset(train_data,time_step)
X_test,y_test = create_dataset(test_data,time_step)

In [ ]:
X_test.shape

In [ ]:
Y_train.shape

In [ ]:
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow import keras

In [ ]:
model = Sequential()
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1)) 
model.compile(loss='mse',optimizer='adam')

In [ ]:
model.fit(X_train,Y_train,epochs=100,verbose=1,validation_data=(X_test,y_test))

In [ ]:
train_prediction = model.predict(X_train)
test_prediction = model.predict(X_test)
test_prediction

In [ ]:
train_prediction = scaler.inverse_transform(train_prediction)
test_prediction = scaler.inverse_transform(test_prediction)
test_prediction

In [ ]:
train_prediction

In [ ]:
plt.plot(train_prediction)

In [ ]:
plt.plot(test_prediction)

In [ ]:
len(test_data)

In [ ]:
x_input=test_data[371:].reshape(1,-1)
x_input.shape


In [ ]:
temp_input = list(x_input)
temp_input = x_input[0].tolist()
len(temp_input)

In [ ]:
lst_output=[]
n_steps=100
i=0
while(i<30):
    
    if(len(temp_input)>100):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
    

print(lst_output)

In [ ]:
day_new=np.arange(1,101)
day_pred=np.arange(101,131)

In [ ]:
x = scaler.inverse_transform(list_loss[1156:])
y = scaler.inverse_transform(lst_output)

In [ ]:
y

In [ ]:
x

In [ ]:
#predicting next 30 days
plt.plot(day_new,scaler.inverse_transform(list_loss[1156:]))
plt.plot(day_pred,scaler.inverse_transform(lst_output))

In [ ]:
df3=list_loss.tolist()
df3.extend(lst_output)
plt.plot(df3[1200:])